# SimPy

Primero procedemos a instalar la librería (Esta no se encuentra en Colab):

El método para instalar en colab difiere de Windows:

`pip install simpy` Para Windows o también: `$ pip install simpy`


`!pip install simpy` Para Colab

`sudo apt install python3-simpy` para la instación en distribuciones GNU-Linux (No en todas)





In [ ]:
!pip install simpy

In [ ]:
import simpy

**Simpy** es una biblioteca de Python utilizada para la simulación de eventos discretos (de ahí su nombre **Sim**ulate **Py**thon). Permite a los usuarios modelar sistemas complejos y simular su comportamiento a lo largo del tiempo, esta libreria es usada en diversas áreas como la ingeniería, la investigación operativa y la gestión de procesos.

In [ ]:
#Un ejemplo
import random
import math

#Definimos la clase Particula
class Particula:
  def __init__(self, env, nombre, x, y, vx, vy, contenedor):
    self.env = env
    self.nombre = nombre
    self.x = x #Posición en x
    self.y = y #Posición en y
    self.vx = vx #Velocidad de x
    self.vy = vy #Velocidad en y
    self.contenedor = contenedor
    self.action = env.process(self.mover())

  def mover(self):
    while True:
      #Calculamos el tiempo en el que realizará el siguiente movimiento
      tiempo = 0.1 # Intervalo de tiempo para el movimiento
      yield self.env.timeout(tiempo)

      #Actualizamos la posición de la partícula
      self.x += self.vx * tiempo
      self.y += self.vy * tiempo

      #Verificamos las colisiones de la partícula en la pared del contenedor
      if self.x <= 0 or self.x >= self.contenedor['ancho']:
        self.vx = -self.vx #Rebote pared vertical
      if self.y <= 0 or self.y >= self.contenedor['alto']:
        self.vy = -self.vy #Rebote pared horizontal

      # Imprimimos la posición actual de la particula
      print(f'Particula {self.nombre} en ({self.x:.2f}, {self.y:.2f}) en tiempo {self.env.now:.2f}')

El anterior código por ejemplo, se simula un sistema simple donde varias partículas se mueven en un espacio bidimensional y rebotan al chocar con las paredes de un contenedor. Esto sirve para particulas de un gas ideal. A continuación se procede a ejecutar los datos que requiramos

In [ ]:
#Colocamos los datos
an = float(input("Ingrese el ancho: "))
al = float(input("Ingrese el alto: "))
n = int(input("Ingrese el numero de particulas: "))
t = float(input("Ingrese el tiempo de simulacion: "))

# Definimos el entorno de simulación
env = simpy.Environment()
contenedor = {'ancho': an, 'alto': al} #dimensiones del contenedor

#Creamos particulas con posiciones y velocidades aleatorias
for i in range(n):
  x = random.uniform(0, contenedor['ancho'])
  y = random.uniform(0, contenedor['alto'])
  vx = random.uniform(-1, 1)
  vy = random.uniform(-1, 1)
  particula = Particula(env, f'Particula {i+1}', x, y, vx, vy, contenedor)

#Iniciamos la simulación
env.run(until=t) #Simulamos durante n unidades de tiempo

Ingrese el ancho: 10
Ingrese el alto: 12
Ingrese el numero de particulas: 4
Ingrese el tiempo de simulacion: 5
Particula Particula 1 en (5.58, 7.96) en tiempo 0.10
Particula Particula 2 en (9.58, 2.25) en tiempo 0.10
Particula Particula 3 en (4.60, 5.57) en tiempo 0.10
Particula Particula 4 en (9.35, 2.54) en tiempo 0.10
Particula Particula 1 en (5.61, 7.99) en tiempo 0.20
Particula Particula 2 en (9.50, 2.19) en tiempo 0.20
Particula Particula 3 en (4.56, 5.62) en tiempo 0.20
Particula Particula 4 en (9.42, 2.54) en tiempo 0.20
Particula Particula 1 en (5.65, 8.02) en tiempo 0.30
Particula Particula 2 en (9.41, 2.12) en tiempo 0.30
Particula Particula 3 en (4.52, 5.67) en tiempo 0.30
Particula Particula 4 en (9.49, 2.53) en tiempo 0.30
Particula Particula 1 en (5.69, 8.04) en tiempo 0.40
Particula Particula 2 en (9.33, 2.06) en tiempo 0.40
Particula Particula 3 en (4.48, 5.72) en tiempo 0.40
Particula Particula 4 en (9.55, 2.52) en tiempo 0.40
Particula Particula 1 en (5.72, 8.07) en 

Los procesos anteriores se describen mediante generadores, es decir, estos si se llegan a pausar no causarán  tantas afectaciones al código, es decir, se puede volver a continuar el proceso desde donde se le pausó. A lo largo de la ejecución, el método de proceso crea eventos y en cierto tiempo el proceso se detiene temporalmente hasta que se cumple la condición del evento, y así  esperar a que ocurran nuevamente ("Predice"). Para ello está los yield, que es lo que permite que un generador pause su ejecución. Cuando el proceso llega a yield, se detiene y devuelve el control al entorno de Simpy, y a su vez, este ejecuta otros procesos, con el fin de reanudar nuevamenete el ciclo.

Otro importente es Timeout que permite que un proceso espere un tiempo simulado antes de continuar su ejecución. Esto es útil para modelar situaciones donde algo debe suceder después de un cierto período de tiempo. Para crear este tipo de evento, se utiliza un método del entorno de Simpy, como `Environment.timeout()`. Esto permite que el proceso "duerma" o mantenga su estado hasta que llegue el momento de reanudar su actividad.

In [ ]:
# Ejemplo

def car(env):
  while True:
    print('Start parking at %d' % env.now)
    parking_duration = 5
    yield env.timeout(parking_duration)

    print('Start driving at %d' % env.now)
    trip_duration = 2
    yield env.timeout(trip_duration)

In [ ]:
env = simpy.Environment()
env.process(car(env))

<Process(car) object at 0x7d7a81053ca0>

In [ ]:
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


Los eventos se organizan por prioridad y tiempo, y pueden tener callbacks y valores de retorno. Los componentes principales son el Entorno (`Environment`), los eventos (`events`) y las funciones de proceso (`simpy.events.Process`), que definen el comportamiento de la simulación. Cuando una función de proceso genera un evento, se suspende hasta que el evento se activa, momento en el cual puede recibir el valor del evento. (En el caso del carro los tiempos en que parquea el carro)

In [ ]:
#Otro Ejemplo
import simpy

def example(env):   #Creamos una instancia para la simulación
  event = simpy.events.Timeout(env, delay=1, value=42) #Se crea el evento tipo Timeout el cual proramará su ejecución a un 1 segundo después de iniciado el programa y un valor asociado a 42
  value = yield event #example va a parar hasta que event se active. Cuando el evento se activa, el valor asociado (42) se asigna a la variable value.
  #"Yield pausará la ejecución de la función hasta que el evento ocurra."
  print('now=%d, value=%d' % (env.now, value))#Cuando el evento se activa y la función se reanuda, se imprime el tiempo actual del entorno y el valor del evento (value), que es 42.

env = simpy.Environment() #Se crea esto para gestionar los eventos y el tiempo de la simulación
example_gen = example(env) #Se crea un generador a partir de la función example, pasando el entorno env como argumento
p = simpy.events.Process(env, example_gen) # Se crea un proceso en el entorno env utilizando el generador example_gen. Example parte de la simulación

env.run()
print()

now=1, value=42



Funciones Importantes de SimPy aparte de `environment` y `event` son:

Para el uso de los recursos que pueden ser utilizados para modelar, sobre todo si son por recursos limitados:
- `Resource`: Un recurso básico que permite a los procesos solicitar y liberar recursos.

- `PriorityResource`: Similar a `Resource`, pero permite la gestión de prioridades entre los procesos que solicitan el recurso.

- `Container`: Utilizado para modelar recursos que tienen una capacidad variable, como el almacenamiento de productos.

`Process` Estos son funciones generadoras que plasman actividades en la simulación. Se definen utilizando `yield` para señalar cuándo un proceso debe esperar (por ejemplo, por un recurso o un evento). Los procesos usan para modelar cualquier cosa que tenga una duración en el tiempo.

`Interrupt`:Se mencionó antes esto, esto interrumpe procesos. Esto es útil para modelar situaciones en las que un proceso puede ser interrumpido por otro evento o proceso, permitiendo una mayores facilidades en la simulación y ahorra recursos.

`timeout`: Permite a los procesos esperar un período de tiempo dado antes de continuar. Sirve para simular retrasos o tiempos de espera.

`RealtimeEnvironment` es una clase en SimPy que se utiliza para simular entornos en tiempo real. A diferencia de un `Environment` estándar, que avanza el tiempo de simulación de manera discreta, `RealtimeEnvironment` permite que la simulación avance en tiempo real, lo que significa que el tiempo de simulación se sincroniza con el tiempo real del sistema.

Es usado en situaciones donde la simulación refleje el tiempo real, como en simulaciones de sistemas que interactúan con el mundo real. Al igual que en un `Environment` estándar, se `yield env.timeout(tiempo)` para simular la duración de eventos, pero en este caso, el tiempo se mide en tiempo real.



In [ ]:
import simpy

def proceso(env):
  print(f'Proceso comienza en {env.now}')
  yield env.timeout(5)  # Simula un proceso que dura 5 segundos
  print(f'Proceso termina en {env.now}')

# Creación de un entorno en tiempo real
env = simpy.RealtimeEnvironment()
env.process(proceso(env))

# Ejecutar la simulación
env.run()


Proceso comienza en 0
Proceso termina en 5
